In [18]:
'''TPC 6 parte 2 trabalho final

OpenCV is in BGR mode. But Matplotlib displays in RGB. A diferença entre opencv e matplotlib é a forma como mostra as cores.
Para a parte da precisão (dados treino e teste) usamos o opencv, e para mostrar os animais usamos o matplotlib.

coisas a instalar linha comandos python:
sudo pip3 install pandas
pip3 install matplotlib
pip3 install opencv-python
pip3 install sklearn
pip3 install tqdm 
pip3 install tflearn
pip3 install tensorflow

Treino tem ficheiros 100 (50 Esquilo + 50 Elefante). Teste tem ficheiros 50 (25 Esquilo + 25 Elefante)
Para efectuarmos o estudo e identificação do tipo de animal renomeamos os ficheiros:
ex:
Esquilo.0.jpeg
Esquilo.1.jpeg
...
Elefante.0.jpeg
Elefante.1.jpeg

#acc is the accuracy (precisão) of a batch of training data and val_acc is the accuracy of a batch of testing
#acc é a precisão dos dados de treino e val_acc é a precisão dos dados de teste
'''

import numpy as np #arrays
import os #diretorias
import cv2 #tratamento do tamanho da imagem
import matplotlib.pyplot as plt #Usado para poder mostrar os animais 
import pandas as pd
from random import shuffle #escolhe aleatoriamente os dados
from tqdm import tqdm #loops - barra de progresso 
from sklearn import datasets, tree, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
%matplotlib inline
# código tflearn- https://pythonprogramming.net/tflearn-machine-learning-tutorial/
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf  #faz o reset dos dados 

tf.reset_default_graph()

image_Size=50 #dimensão imagem (pixeis 50x50)
 
learning_Rate=1e-3 #0.001 permite determinar valores com mais precisão no universo em estudo (MAX,MIN)

path_Directory_Treino="C:/animals10/Treino" #amostra de treino
path_Directory_Teste="C:/animals10/Teste"  #amostra do teste

#padronizar o modelo quanto à formatação e para 6 layers
model_Identification="animals_Esquilo_Elefante-{}-{}.model".format(learning_Rate,"6conv-basic-video")
#"2conv-basic-video" Layers p/ 1 - linear | >=2 - não linear

#com estas funções vamos determinar a probabilidade da imagem ser Esquilo ou Elefante
#Funcao para determinar a imagem(label Esquilo ou Elefante) na path_Directory
def label_image(image):
    animal_label=image.split(".")[-3] # exemplo: Esquilo.0.jpeg
    if animal_label=="Esquilo": return 1
    elif animal_label=="Elefante": return 0

#Vamos determinar a precisão Accuracy pela Funcao de treino e a função de teste utilizada com base na primeira.
#Funcao de treino apenas precisa de ser executa 1 primeira vez, temos as imagens/features e a labels servem para determinar a precisão(accuracy)
def create_Treino():
	treino_data=[] #Iniciação lista vazia
	for image in tqdm(os.listdir(path_Directory_Treino)):
		label=label_image(image)
		path=os.path.join(path_Directory_Treino,image) #Junta o path com a imagem
		image=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(image_Size,image_Size)) # padronizar o tamanho, e ler imagens a cinzento
		treino_data.append([np.array(image),label]) # adicionar no array com o label e imagem num formato numpy
		shuffle(treino_data) #mistura os dados treino
		np.save('treino_data.npy',treino_data) # guarda os dados resultante do treino
	return treino_data # retorna os dados


#Funcao de Teste ajuda a determinar a accuracy pela utilizacao dos dados de treino sem a existência de labels
def execucao_Teste():
	teste_data=[]
	for image in tqdm(os.listdir(path_Directory_Teste)):
		path=os.path.join(path_Directory_Teste,image) #Junta o path com a imagem
		image_id=image.split('.')[0]
		image=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(image_Size,image_Size)) # padronizar o tamanho, e ler imagens a cinzento
		teste_data.append([np.array(image),image_id]) #Associa imagem ao id
	np.save('teste_data.npy',teste_data)
	return teste_data

#Criar os dados de treino
treino_data=create_Treino()

# código tflearn
convnet = input_data(shape=[None, image_Size, image_Size, 1], name='input')# tamanho de imagem definido em image_Size

#6 layers (aumenta-se nr layers p melhorar a precisão)
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation='softmax') # 2:  Esquilo ou Elefante
convnet = regression(convnet, optimizer='adam', learning_rate=learning_Rate, loss='categorical_crossentropy', name='targets')# Definido learning_Rate
print(convnet)
model = tflearn.DNN(convnet,tensorboard_dir='log') #cria registos de actividade



#carrega o existente
treino_data=np.load('treino_data.npy')
teste_data=np.load('teste_data.npy')

#Determinar a Accuracy entre os dados de treino e de teste P/ accuracy semelhante
treino=treino_data[:100] #100 exemplos de dados
teste= treino_data[-50:] #usa os ultimos 50

#standardizar ou seja converter em numpy array as imagens e os labels 
# X: feature sets Y: label sets
X=np.array([i[0] for i in treino]).reshape(-1,image_Size,image_Size,1) #i[0] posição com as imagens e dimensionamento das imagens
Y=[i[1] for i in treino]

#Testar a accuracy de X e Y anteriores. A diferença é que é de teste (accuracy), enquanto o de cima é de treino (fit)
TesteX=np.array([i[0] for i in teste]).reshape(-1,image_Size,image_Size,1) #i[0] posição com as imagens e dimensionamento das imagens
TesteY=[i[1] for i in teste]

#A informação vai ser inserida no modelo
model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': TesteX}, {'targets': TesteY}), #n_epoch=Training step
   snapshot_step=50, show_metric=True, run_id='modeloEstudo')


#Identificar os animais
teste_data=execucao_Teste()
fig=plt.figure()
for id,dataAnimal in enumerate(treino_data[:12]):#Vou mostrar 12 animais
    #esquilo: [1, 0]
    #elefante: [0, 1]
    image_id=dataAnimal[1]
    data_image=dataAnimal[0]
    y=fig.add_subplot(3,4,id+1)#mostra a matriz com as imagens 4x3 com incremento do id
    source_image=data_image
    dataAnimal=data_image.reshape(image_Size,image_Size,1) #padronizar o tamanho das imagens
    modelo=model.predict([dataAnimal])[0]# lista de 1 prediction (previsão)

    if np.argmax(modelo)==1: label_animal="Elefante"
    else: label_animal="Esquilo"
    
    y.imshow(source_image,cmap="gray")# para a imagem mostrar em cinzento
    plt.title(label_animal)
    y.axes.get_xaxis().set_visible(False) #Não mostra o eixo do x
    y.axes.get_yaxis().set_visible(False) #Não mostra o eixo do y
plt.show()# Mostra a imagem e o titulo


##################################################################################
#Write e Append csv
with open('submeter_ficheiro.csv', 'w') as f:
    f.write('id, label\n')
 
with open('submeter_ficheiro.csv', 'a') as f:
    for data in tqdm(teste_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(image_Size, image_Size, 1)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))
        
        

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 169.87it/s]


Tensor("FullyConnected_1/Softmax:0", shape=(?, 2), dtype=float32)
---------------------------------
Run id: modeloEstudo
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 100
Validation samples: 50
--


ValueError: Cannot feed value of shape (64,) for Tensor 'targets/Y:0', which has shape '(?, 2)'

In [7]:
########################
TesteX.shape

(50, 50, 50, 1)

X.shape

(100, 50, 50, 1)

In [5]:
X.reshape(100, -1,1).squeeze().shape

(100, 2500)

In [19]:
len(Y)
Y[0]

1

In [15]:
classes_train

[1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1]

In [14]:
#Decision Tree

clf = tree.DecisionTreeClassifier()
[features_train, features_test, classes_train, classes_test] = model_selection.train_test_split(X.reshape(100, -1,1).squeeze(), Y, test_size=0.30)
model = tree.DecisionTreeClassifier()
clf = model.fit(features_train, classes_train)
score_train = model.score(features_train, classes_train)
score_test = model.score(features_test, classes_test)
print ("Decision Tree")
print("score_train:", score_train)
print("score_test:", score_test)
print("--------------------------------------------")


#Random Forest 
[features_train, features_test, classes_train, classes_test] = model_selection.train_test_split(X.reshape(100, -1,1).squeeze(), Y, test_size=0.30)

model = RandomForestClassifier(n_estimators=100)
clf = model.fit(features_train, classes_train)
clf
score_train = model.score(features_train, classes_train)
score_test = model.score(features_test, classes_test)
print ("Random Forest")
#print("Features:", digits.target_names)
print("score_train:", score_train)
print("score_test:", score_test)
print("--------------------------------------------")

Decision Tree
score_train: 1.0
score_test: 0.5
--------------------------------------------
Random Forest
score_train: 1.0
score_test: 0.7333333333333333
--------------------------------------------
